In [1]:
# Entregatzeko hobeto import guztiak batera jarri...
import collections
import random

import community
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [2]:
from ipynb.fs.defs.CDP_Sarrera_Ikasle import sortu_grafoa 

G = sortu_grafoa()
print(len(G), 'nodo ditu.')

1843 nodo ditu.


**`lortu_kom_auzokidea`**

Nodo baten izena, uneko soluzioa eta grafoa emanda, auzokideen komunitateen lista bat itzultzen du, komunitateak errepikatu gabe eta erpinaren uneko komunitatea sartu gabe.

In [3]:
def lortu_kom_auzokideak(izena, soluzioa, G):
    komunitateak = set()
    for auzokidea in G[izena]:
        komunitatea = soluzioa[auzokidea]
        if komunitatea != soluzioa[izena]:
            komunitateak.add(soluzioa[auzokidea])
    return list(komunitateak)

**`lortu_soluzio_auzokide_bat`**

Uneko soluzioa, grafoa eta erpinen izenen lista bat behar ditu (azken hau soluziotik har zitekeen, baina horrela ez da aldiro sortu behar). Soluzioa aldatzen du ausazko auzokide batekin. Aukeratutako erpinaren izena eta aurretik esleituta zeukan komunitatea itzultzen ditu, egindako aldaketa desegin ahal izateko, saiakera onartzen ez bada. Horrela, saiakera bakoitzeko soluzioaren kopia bat sortzea ekiditen da.

In [4]:
def lortu_soluzio_auzokide_bat(soluzioa, G, izen_lista, k):
    komunitateak = []
    nodoak = izen_lista.copy()
    random.shuffle(nodoak)
    for n in nodoak:
        izena = n
        komunitateak = lortu_kom_auzokideak(izena, soluzioa, G)
        if komunitateak != []:
            break

    if komunitateak != []:
        komunitatea = random.choice(komunitateak)
    # Beste komunitate bateko auzokideren bat daukan erpinik ez dagoenean, ausaz erpin bat hartu 
    # eta bere auzokideekin batera beste komunitate batera mugitzen dira 
    # (komunitate kopurua oso txikia denean batzuetan gertatzen da)
    else:
        izena = random.choice(izen_lista)
        lista = list(range(k))
        lista.remove(soluzioa[izena])
        komunitatea = random.choice(lista)

    kom_zaharra = soluzioa[izena]
    soluzioa[izena] = komunitatea
    return izena, kom_zaharra

**`batez_besteko_aldaketa`**

Hasierako tenperatura ezartzeko prozesuan erabiltzen da. Hasierako soluzioa, bere modularitatea, grafoa, erpinen izen-lista, komunitate-kopurua eta egin beharreko saiakera-kopurua emanda, ausazko auzokideen modularitatearen batez besteko diferentzia kalkulatzen du, okerrera egiten duten soluzioak bakarrik hartuta kontuan.

In [5]:
def batez_besteko_aldaketa(soluzioa, modul, G, izen_lista, komunitate_kop, saiakera_kop):
    oker_kop = 0
    batura = 0
    for _ in range(saiakera_kop):
        izena, kom_zaharra = lortu_soluzio_auzokide_bat(soluzioa, G, izen_lista, komunitate_kop)
        emaitza = community.modularity(soluzioa, G)
        soluzioa[izena] = kom_zaharra
        if emaitza < modul:
            oker_kop += 1
            batura += modul - emaitza
    batezbeste = batura / oker_kop
    return batezbeste

**`onartua`**

Aukera berriaren energia, uneko soluzioarena eta tenperatura emanda, aukera onartuko den ala ez itzultzen du.

In [6]:
def onartua(E_berria, E_unekoa, T):
    if E_berria < E_unekoa:
        return True
    else:
        p = np.exp((E_unekoa-E_berria) / T)
        return p > random.random()

**`simulated_annealing`**

Funtzio nagusia. Parametroak hauek dira: grafoa, komunitate-kopurua, hasierako soluzioa, hasieran soluzio txar bat onartzeko probabilitatea, hasierako tenperatura kalkulatzeko egin daitezkeen saiakeren kopurua, guztira egin daitezkeen saiakeren kopurua (modularitatea zenbat aldiz kalkulatu), hozte-faktorea eta epoka-luzera. Aurkitutako soluzio onena eta bere modularitatea itzultzen ditu.

In [7]:
def simulated_annealing(G, komunitate_kop, sol_hasi, p0, hasierako_kop, budget, hozte_faktorea, epoka_luzera):
    sol_unekoa = sol_hasi
    izen_lista = list(G.nodes())
    mod_unekoa = community.modularity(sol_unekoa, G)
    mod_onena = mod_unekoa
    deltaE = batez_besteko_aldaketa(sol_unekoa, mod_unekoa, G, izen_lista, komunitate_kop, hasierako_kop)
    T = -deltaE/np.log(p0)
    bukatu = False
    saiakerak = hasierako_kop + 1
    while not bukatu:
        hobe_kop = 0
        onartu_kop = 0
        for _ in range(epoka_luzera):
            izena, kom_zaharra = lortu_soluzio_auzokide_bat(sol_unekoa, G, izen_lista, komunitate_kop)
            mod_berria = community.modularity(sol_unekoa, G)
            if onartua(-mod_berria, -mod_unekoa, T):
                if mod_berria > mod_unekoa:
                    hobe_kop += 1
                else:
                    onartu_kop += 1
                mod_unekoa = mod_berria
                if mod_unekoa > mod_onena:
                    mod_onena = mod_unekoa
                    sol_onena = sol_unekoa.copy()
            else:
                sol_unekoa[izena] = kom_zaharra
            saiakerak += 1
            if saiakerak >= budget:
                bukatu = True
                break
        T *= hozte_faktorea
        
    return mod_onena

**`eraikitzailea`**

Grafoa eta komunitate-kopurua emanda, soluzio bat eraiki eta itzultzen du.

In [8]:
def eraikitzailea(G, komunitate_kop):
    kom_lista = len(G) * [-1]
    soluzioa = dict(zip(G.nodes, kom_lista))
    ilara = collections.deque()
    komunitatea = 0
    min_kop = len(G) // komunitate_kop
    hondarra = len(G) % komunitate_kop
    sartzeko_kop = min_kop + int(komunitatea < hondarra)
    sartu_kop = 0
    izenak = list(G.nodes)
    random.shuffle(izenak)
    for izena in izenak:
        ilara.append(izena)
        while ilara:
            unekoa = ilara.popleft()
            if soluzioa[unekoa] == -1:
                soluzioa[unekoa] = komunitatea
                sartu_kop += 1
                if sartu_kop == sartzeko_kop:
                    komunitatea += 1
                    sartzeko_kop = min_kop + int(komunitatea < hondarra)
                    sartu_kop = 0
                for auzokoa in G[unekoa]:
                    ilara.append(auzokoa)
    return soluzioa

Grid search, 10 komunitate:

In [51]:
import  time
G = sortu_grafoa()
komunitate_kop = 10

rand_lista = list(np.random.randint(komunitate_kop, size=len(G)))
sol_hasi = eraikitzailea(G, komunitate_kop)    
hasierako_kop = 500
budget = 10_000

rep = 10
epoka_luzera = int(budget / 5)

for p0 in [0.1,0.3,0.5,0.7,0.9]:
    for hozte_faktorea in [0.1,0.3,0.5,0.7,0.9]:
        m=0
        for _ in range(rep):
            m += simulated_annealing(G, komunitate_kop, sol_hasi, p0, hasierako_kop, budget, hozte_faktorea, epoka_luzera)
        print(p0, hozte_faktorea, m/rep)
bukatu = time.time()

0.1 0.1 0.8862049882552772
0.1 0.3 0.8872168053894992
0.1 0.5 0.8884850724555531
0.1 0.7 0.8865153821630276
0.1 0.9 0.8863350746999764
0.3 0.1 0.887275279168574
0.3 0.3 0.8872054767859467
0.3 0.5 0.8873414241242408
0.3 0.7 0.8872753692731574
0.3 0.9 0.8855334346982595
0.5 0.1 0.8823342878412219
0.5 0.3 0.8852105654013471
0.5 0.5 0.8864552537361456
0.5 0.7 0.8849043204001037
0.5 0.9 0.8764666445937742
0.7 0.1 0.883232511765201
0.7 0.3 0.8855421871298546
0.7 0.5 0.8840101388953965
0.7 0.7 0.8765986232347283
0.7 0.9 0.8362279690197505
0.9 0.1 0.8800583648344418
0.9 0.3 0.8817201431909277
0.9 0.5 0.8659755877194334
0.9 0.7 0.8198973856236991
0.9 0.9 0.7838909714879609


Grid search, 20 komunitate:

In [52]:
import  time
G = sortu_grafoa()
komunitate_kop = 20

rand_lista = list(np.random.randint(komunitate_kop, size=len(G)))
sol_hasi = eraikitzailea(G, komunitate_kop)    
hasierako_kop = 500
budget = 10_000

rep = 10
epoka_luzera = int(budget / 5)

for p0 in [0.1,0.3,0.5,0.7,0.9]:
    for hozte_faktorea in [0.1,0.3,0.5,0.7,0.9]:
        m=0
        for _ in range(rep):
            m += simulated_annealing(G, komunitate_kop, sol_hasi, p0, hasierako_kop, budget, hozte_faktorea, epoka_luzera)
        print(p0, hozte_faktorea, m/rep)
bukatu = time.time()

0.1 0.1 0.932801652124879
0.1 0.3 0.9363175985064263
0.1 0.5 0.935790777484601
0.1 0.7 0.93650810007441
0.1 0.9 0.9352681299432604
0.3 0.1 0.9292519247158182
0.3 0.3 0.931547293929998
0.3 0.5 0.9305924065099414
0.3 0.7 0.9305483781338783
0.3 0.9 0.9279361725342226
0.5 0.1 0.9278063850729144
0.5 0.3 0.9272104374528384
0.5 0.5 0.9295393460504211
0.5 0.7 0.9281035991374861
0.5 0.9 0.9176822258715488
0.7 0.1 0.9232783799130342
0.7 0.3 0.9252807576910815
0.7 0.5 0.9275314759884392
0.7 0.7 0.9176014225383999
0.7 0.9 0.9053473384252797
0.9 0.1 0.921576439486253
0.9 0.3 0.9237658457101533
0.9 0.5 0.9136080488294759
0.9 0.7 0.8624654367008748
0.9 0.9 0.8150965437847659


Probak 51 komunitatetik 100 komunitatera (2tik 50era beste notebook batean egin dira):

In [21]:

G = sortu_grafoa()
   
p0 = 0.1
hasierako_kop = 500
budget = 10_000
hozte_faktorea = 0.5

rep = 10
epoka_luzera = int(budget / 5)

for komunitate_kop in range(51,101):
    modularitatea = []
    for _ in range(rep):
        sol_hasi = eraikitzailea(G, komunitate_kop) 
        m = simulated_annealing(G, komunitate_kop, sol_hasi, p0, hasierako_kop, budget, hozte_faktorea, epoka_luzera)
        modularitatea.append(m)
        print(m)
    print(komunitate_kop, (np.sum(modularitatea)/rep))

0.9371984977763826
0.9472844362516626
0.940785151682056
0.9374343260455656
0.9386391881544257
0.9454731702953696
0.9427001607793426
0.9362378600456291
0.9394798229625142
0.9405841365474347
51 0.9405816750540381
0.9384417362465183
0.9375234067134142
0.9402618897912817
0.9426309031198798
0.9406884530812325
0.9385255744658682
0.9411505257684364
0.9439268527714042
0.9438781143830179
0.9389429634709464
52 0.9405970419812
0.9367864740896898
0.940346792882983
0.9422187565632996
0.9408910655243807
0.9355379522144267
0.9412434973160305
0.9378837840910694
0.9365207474814132
0.939730477531357
0.9398805016630031
53 0.9391040049357653
0.9240456859727479
0.9360181277316021
0.9409841599418614
0.9433534190102324
0.943309185851025
0.9375833672181175
0.9357803335442326
0.9419164966420478
0.9392410048594223
0.9381748218960038
54 0.9380406602667293
0.9350010927228587
0.9460241598239063
0.946084407025012
0.9375422467627063
0.9423186497811767
0.9365008835163985
0.9385174650533467
0.9390696423241218
0.940297

0.9387691435379123
0.9362521948657426
0.9355116990153043
0.9496349781406282
0.9398664944959204
0.9378011745705866
0.9392182739303852
0.9338000805371152
0.9304776378444497
0.9388581422925031
90 0.9380189819230548
0.9421055933976604
0.9387373611938891
0.9293806964527299
0.9397824105367973
0.9410168433317201
0.9372031258754482
0.9380398698037916
0.9385311855240269
0.9268092754641292
0.9374754055443483
91 0.936908176712454
0.9401542557705437
0.9325549171054215
0.9393777590433056
0.9401049439893528
0.9364960915908178
0.9438332259177482
0.9411913595274455
0.9381728559778163
0.9407555400393648
0.9444045299342174
92 0.9397045478896034
0.9443906456370217
0.9472412679648062
0.9442425464669343
0.9347978249736488
0.9378347590062805
0.9358526219942338
0.9356834301602652
0.9349551803418668
0.9408460951458533
0.9351603320960062
93 0.9391004703786916
0.933414719615834
0.9432292794680489
0.9340239085140314
0.9385806201751045
0.9337620318288707
0.9402026255492691
0.9326346187052532
0.9352920486145357
0.

Eraikitzailearekin bakarrik egindako probak (SA gabe):

In [9]:
G = sortu_grafoa()

rep = 10

for komunitate_kop in range(2,101):
    modularitatea = []
    for _ in range(rep):
        soluzioa = eraikitzailea(G, komunitate_kop) 
        m = community.modularity(soluzioa, G)
        modularitatea.append(m)
        print(m)
    print(komunitate_kop, (np.sum(modularitatea)/rep))

0.4801472980584265
0.47268848158704646
0.4796391491637147
0.47945566346626106
0.4709632245514348
0.4695480501531941
0.46706824914343303
0.4694171937238718
0.45997750989594066
0.47772123214577666
2 0.47266260518891007
0.641257119490801
0.6427344660517056
0.6303920221074053
0.6455991774598305
0.650564595320656
0.6430131349547135
0.63938011814513
0.6457888885648765
0.6404649773313251
0.6498147613588295
3 0.6429009260785273
0.7196168572897393
0.7101724224928564
0.7194736319584902
0.7114541192375252
0.7158834967393609
0.7171361142775136
0.7172126622169199
0.7160470775151712
0.7123523800224705
0.7172364989749376
4 0.7156585260724985
0.7598026414732066
0.7636983950571574
0.7581761308698631
0.7637610177427393
0.7678239153292143
0.7665468876402397
0.7696717555551114
0.7618777090762182
0.76566808125402
0.7587378510351706
5 0.763576438503294
0.7937054740008959
0.7914425383411386
0.7981508245880009
0.7854069237672628
0.7874210888631062
0.7898469090359825
0.7801258941241654
0.7890563641850334
0.796

0.7504358604447103
0.7453697302334297
0.7700649801491412
0.7756952469668339
0.7388813810837384
0.7773260170595827
0.7739406240283042
41 0.7609514667224113
0.7418358284657744
0.7775861735663295
0.7513486198762845
0.7735063608921209
0.752866636369683
0.7538590154878313
0.7862926518237661
0.7698079363461905
0.7631750922097544
0.7592419042669927
42 0.7629520219304728
0.7760725803888522
0.7744261239072362
0.7595487103740518
0.7491123879385678
0.7656105371904214
0.7443519170487375
0.7614334934792132
0.7678680256185351
0.7722725014900023
0.749283545690724
43 0.7619979823126342
0.7488800409828409
0.7756916018268621
0.7565216878456126
0.7627180162312758
0.7572612826502152
0.7540905433109796
0.7590248341338441
0.7536992846351371
0.7639304143598526
0.7648199513828433
44 0.7596637657359462
0.7656445721490337
0.7459260031670942
0.7430848008639555
0.7510888729358269
0.7565699347544519
0.7745078323818835
0.7609660145169953
0.7464129365280379
0.7901274209872906
0.782921798395745
45 0.7617250186680314


0.7136087246138528
0.7003760883404998
0.7211995033435353
0.6912922357044162
0.7189768280499832
80 0.7075483845231125
0.6994106586841654
0.6992032543154365
0.7055931846858905
0.7148488503801922
0.6988377573591689
0.707362265314421
0.6990755105899102
0.7013103500022443
0.7226096810330503
0.6960724476893745
81 0.7044323960053853
0.7153027726982456
0.7057290787805673
0.6788992103234296
0.7220078233714249
0.6934539675341719
0.7118174454924608
0.6934482336061261
0.7292289275686935
0.7048789010779458
0.7034951404140684
82 0.7058261500867133
0.7000934876011179
0.7050736908049746
0.7020588733519463
0.6835055204620957
0.7202170766862087
0.7096976942073242
0.7013604809160132
0.6926107934150918
0.6901906252570031
0.7102120275529985
83 0.7015020270254774
0.7050914250252867
0.6956188530243521
0.6938011159206934
0.70215888943971
0.6875433525916865
0.6983412401470639
0.7104810306915872
0.7054073644605898
0.7017673031108366
0.6922027015647728
84 0.6992413275976578
0.6922210091778895
0.7010860305457814
